In [129]:
# Import libraries
import os
import boto3
import pandas as pd
import sys
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import psycopg2
import sqlalchemy
import matplotlib as plt
import numpy as np
%matplotlib inline

In [2]:
# Get the S3 keys from config file
from config import aws_id, aws_secret, postgres_key

In [3]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import psycopg2
import pandas.io.sql as sqlio

In [4]:
# Upload fec_df to sql database
conn = psycopg2.connect(f"host=projectdatabase.c1ibogbs35nd.us-east-2.rds.amazonaws.com dbname=election_results user=postgres password={postgres_key}")
cur = conn.cursor()
cur.execute("SELECT state_district, name, party, incumbent_challenger_open, receipts, from_individuals, from_committees, from_candidate, disbursements, cash_on_hand, debts from fec_2018")
#with open("Resources/fec.csv", "r") as f:
 #   next(f) # Skip the header row.
  #  cur.copy_from(f, "fec", sep=",")
#conn.commit()

rows = cur.fetchall()
rows

[('AL01',
  'BYRNE BRADLEY ROBERTS',
  'Republican Party',
  'Incumbent',
  1001925.17,
  200181.59,
  742250.0,
  0.0,
  533325.02,
  914918.48,
  0.0),
 ('AL01',
  'KENNEDY ROBERT JR.',
  'Democratic Party',
  'Challenger',
  133664.39,
  13664.39,
  0.0,
  120000.0,
  17053.18,
  116611.21,
  120000.0),
 ('AL01',
  'MCCONNELL LIZZETTA HILL',
  'Democratic Party',
  'Challenger',
  5127.0,
  5127.0,
  0.0,
  0.0,
  6021.0,
  -894.0,
  0.0),
 ('AL02',
  'ROBY MARTHA',
  'Republican Party',
  'Incumbent',
  2186212.91,
  718837.48,
  1381291.24,
  0.0,
  1500989.88,
  730673.75,
  0.0),
 ('AL02',
  'AMASON THOMAS MARRION III',
  'Republican Party',
  'Challenger',
  15380.59,
  8366.4,
  0.0,
  2014.19,
  14007.45,
  -2516.19,
  0.0),
 ('AL02',
  'BRIGHT BOBBY NEAL SR.',
  'Republican Party',
  'Challenger',
  406557.48,
  96077.48,
  1000.0,
  309480.0,
  243958.88,
  162598.6,
  309480.0),
 ('AL02',
  'HOBSON RICH',
  'Republican Party',
  'Challenger',
  76307.05,
  75807.05,
  0.0,

In [5]:
# Connect to final table in SQL
sql = "SELECT state_district, election, name, party, incumbent_challenger_open, receipts, from_individuals, from_committees, from_candidate, disbursements, cash_on_hand, debts, district, election_year, total_pop, male_pop_percent, female_pop_percent, under_five_percent, five_to_nine_percent,ten_to_fourteen_percent,fifteen_nineteen_percent,twenty_to_twentyfour_percent,twentyfive_to_thirtyfour_percent ,thirtyfive_to_fortyfour_percent ,fortyfive_to_fiftyfour_percent ,fiftyfive_to_fiftynine_percent ,sixty_to_sixtyfour_percent ,sixtyfive_to_seventyfour_percent ,seventyfive_to_eightyfour_percent ,over_eightyfive_percent ,median_age, eighteen_and_over_percent ,twentyone_and_over_percent ,sixtytwo_and_over_percent ,sixtyfive_and_over_percent ,eighteen_and_over_male_percent ,eighteen_and_over_female_percent ,sixtyfive_and_over_male_percent ,sixtyfive_and_over_female_percent ,white_pop_percent ,black_pop_percent ,american_indian_alaska_native_pop_percent ,asian_pop_percent ,native_hawaiian_pacific_islander_pop_percent ,other_race_pop_percent ,hispanic_pop_percent , total_housing_units, last_name, first_name, prefix, first_last_name, district_from_length, candidate_name, spent, won, lost, incumbent, challenger, open_seat, election_year_from_length, party_from_length, year, Last_name, firstlast_name from final_table_1"
final_summary = sqlio.read_sql_query(sql, conn)
final_summary

,state_district,election,name,party,incumbent_challenger_open,receipts,from_individuals,from_committees,from_candidate,disbursements,...,won,lost,incumbent,challenger,open_seat,election_year_from_length,party_from_length,year,last_name,firstlast_name
0,IL06,2014.0,ROSKAM PETER,Republican Party,Incumbent,2643875.43,1137871.17,1443315.73,0.00,2543254.32,...,1,0,1,0,0,2014,Republican,2007.0,Roskam,Peter Roskam
1,TX05,2014.0,HENSARLING JEB HON.,Republican Party,Incumbent,2673006.24,1384157.24,1234898.00,0.00,3084464.90,...,1,0,1,0,0,2014,Republican,2003.0,Hensarling,Jeb Hensarling
2,LA01,2014.0,SCALISE STEVE MR.,Republican Party,Incumbent,1425973.57,567158.50,854339.38,0.00,1168222.75,...,1,0,1,0,0,2014,Republican,2008.0,Scalise,Steve Scalise
3,NY14,2014.0,CROWLEY JOSEPH,Democratic Party,Incumbent,2522320.53,623251.17,1286586.77,0.00,1946781.49,...,1,0,1,0,0,2014,Democrat,1999.0,Crowley,Joseph Crowley
4,TX08,2014.0,BRADY KEVIN,Republican Party,Incumbent,2018388.20,439001.78,1223160.02,0.00,1010201.52,...,1,0,1,0,0,2014,Republican,1997.0,Brady,Kevin Brady
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4872,CA34,2018.0,CARMONA ARTURO,Democratic Party,Open,155331.16,142631.16,9700.00,3000.00,244941.43,...,0,1,0,0,0,0,0,0.0,Carmona,0
4873,TX21,2018.0,POTEET ALONZO MARCUS MR. III,Republican Party,Open,27518.18,6795.67,0.00,20722.51,30835.17,...,0,1,0,0,0,0,0,0.0,Poteet,0
4874,WA08,2018.0,MARTS TOLA,Democratic Party,Open,74353.87,73493.85,0.00,860.02,74353.87,...,0,1,0,0,0,0,0,0.0,Marts,0
4875,OH02,2018.0,EVERHARD JANET,Democratic Party,Challenger,57777.75,29205.37,500.00,28072.38,54737.28,...,0,1,0,0,0,0,0,0.0,Everhard,0


In [6]:
# Look at spent column of final table
final_summary['spent']

0       $4,079,869 
1       $3,408,251 
2       $2,735,431 
3       $2,668,960 
4       $2,184,911 
           ...     
4872              0
4873              0
4874              0
4875              0
4876              0
Name: spent, Length: 4877, dtype: object

In [7]:
# Take away $ and , from spent column
# Change the spring spent column to a number
final_summary['spent'] = final_summary['spent'].str.replace(',', '')
final_summary['spent'] = final_summary['spent'].str.replace('$', '')
final_summary['spent'] = final_summary['spent'].astype(int)
final_summary['spent']

0       4079869
1       3408251
2       2735431
3       2668960
4       2184911
         ...   
4872          0
4873          0
4874          0
4875          0
4876          0
Name: spent, Length: 4877, dtype: int32

In [8]:
final_summary.head()

,state_district,election,name,party,incumbent_challenger_open,receipts,from_individuals,from_committees,from_candidate,disbursements,...,won,lost,incumbent,challenger,open_seat,election_year_from_length,party_from_length,year,last_name,firstlast_name
0,IL06,2014.0,ROSKAM PETER,Republican Party,Incumbent,2643875.43,1137871.17,1443315.73,0.0,2543254.32,...,1,0,1,0,0,2014,Republican,2007.0,Roskam,Peter Roskam
1,TX05,2014.0,HENSARLING JEB HON.,Republican Party,Incumbent,2673006.24,1384157.24,1234898.00,0.0,3084464.90,...,1,0,1,0,0,2014,Republican,2003.0,Hensarling,Jeb Hensarling
2,LA01,2014.0,SCALISE STEVE MR.,Republican Party,Incumbent,1425973.57,567158.50,854339.38,0.0,1168222.75,...,1,0,1,0,0,2014,Republican,2008.0,Scalise,Steve Scalise
3,NY14,2014.0,CROWLEY JOSEPH,Democratic Party,Incumbent,2522320.53,623251.17,1286586.77,0.0,1946781.49,...,1,0,1,0,0,2014,Democrat,1999.0,Crowley,Joseph Crowley
4,TX08,2014.0,BRADY KEVIN,Republican Party,Incumbent,2018388.20,439001.78,1223160.02,0.0,1010201.52,...,1,0,1,0,0,2014,Republican,1997.0,Brady,Kevin Brady


In [9]:
final_summary.columns

Index(['state_district', 'election', 'name', 'party',
       'incumbent_challenger_open', 'receipts', 'from_individuals',
       'from_committees', 'from_candidate', 'disbursements', 'cash_on_hand',
       'debts', 'district', 'election_year', 'total_pop', 'male_pop_percent',
       'female_pop_percent', 'under_five_percent', 'five_to_nine_percent',
       'ten_to_fourteen_percent', 'fifteen_nineteen_percent',
       'twenty_to_twentyfour_percent', 'twentyfive_to_thirtyfour_percent',
       'thirtyfive_to_fortyfour_percent', 'fortyfive_to_fiftyfour_percent',
       'fiftyfive_to_fiftynine_percent', 'sixty_to_sixtyfour_percent',
       'sixtyfive_to_seventyfour_percent', 'seventyfive_to_eightyfour_percent',
       'over_eightyfive_percent', 'median_age', 'eighteen_and_over_percent',
       'twentyone_and_over_percent', 'sixtytwo_and_over_percent',
       'sixtyfive_and_over_percent', 'eighteen_and_over_male_percent',
       'eighteen_and_over_female_percent', 'sixtyfive_and_over_male_pe

In [10]:
# 'last_name', 'first_name', 'prefix', 'first_last_name', 'district_from_length', 'candidate_name', 'incumbent', 
# 'challenger', 'open_seat', 'election_year_from_length', 'party_from_length', 'Last_name', 'firstlast_name'

# Took out just for presentation , 'name' / 'state_district'

In [11]:
# Drop unnecessary columns for machine learning
final_summary_drop_columns = final_summary.drop(columns=['election_year','last_name', 'first_name', 'prefix', 'first_last_name', 'district_from_length', 'candidate_name', 'incumbent', 'challenger', 'open_seat', 'election_year_from_length', 'party_from_length','district', 'state_district', 'name'])
final_summary_drop_columns.head()

,election,party,incumbent_challenger_open,receipts,from_individuals,from_committees,from_candidate,disbursements,cash_on_hand,debts,...,asian_pop_percent,native_hawaiian_pacific_islander_pop_percent,other_race_pop_percent,hispanic_pop_percent,total_housing_units,spent,won,lost,year,firstlast_name
0,2014.0,Republican Party,Incumbent,2643875.43,1137871.17,1443315.73,0.0,2543254.32,1143860.44,15607.71,...,8.9,0.0,0.1,10.0,270685.0,4079869,1,0,2007.0,Peter Roskam
1,2014.0,Republican Party,Incumbent,2673006.24,1384157.24,1234898.00,0.0,3084464.90,1472329.77,0.00,...,1.8,0.1,0.1,27.9,286382.0,3408251,1,0,2003.0,Jeb Hensarling
2,2014.0,Republican Party,Incumbent,1425973.57,567158.50,854339.38,0.0,1168222.75,858641.66,0.00,...,2.0,0.0,0.2,8.8,330578.0,2735431,1,0,2008.0,Steve Scalise
3,2014.0,Democratic Party,Incumbent,2522320.53,623251.17,1286586.77,0.0,1946781.49,1364191.18,0.00,...,16.0,0.0,0.4,49.8,262554.0,2668960,1,0,1999.0,Joseph Crowley
4,2014.0,Republican Party,Incumbent,2018388.20,439001.78,1223160.02,0.0,1010201.52,1325444.21,0.00,...,2.2,0.0,0.0,22.4,290582.0,2184911,1,0,1997.0,Kevin Brady


In [12]:
final_summary_drop_columns.columns

Index(['election', 'party', 'incumbent_challenger_open', 'receipts',
       'from_individuals', 'from_committees', 'from_candidate',
       'disbursements', 'cash_on_hand', 'debts', 'total_pop',
       'male_pop_percent', 'female_pop_percent', 'under_five_percent',
       'five_to_nine_percent', 'ten_to_fourteen_percent',
       'fifteen_nineteen_percent', 'twenty_to_twentyfour_percent',
       'twentyfive_to_thirtyfour_percent', 'thirtyfive_to_fortyfour_percent',
       'fortyfive_to_fiftyfour_percent', 'fiftyfive_to_fiftynine_percent',
       'sixty_to_sixtyfour_percent', 'sixtyfive_to_seventyfour_percent',
       'seventyfive_to_eightyfour_percent', 'over_eightyfive_percent',
       'median_age', 'eighteen_and_over_percent', 'twentyone_and_over_percent',
       'sixtytwo_and_over_percent', 'sixtyfive_and_over_percent',
       'eighteen_and_over_male_percent', 'eighteen_and_over_female_percent',
       'sixtyfive_and_over_male_percent', 'sixtyfive_and_over_female_percent',
       'wh

In [13]:
# Drop first and last name column
final_summary_drop_columns = final_summary_drop_columns.drop(columns=['firstlast_name'])
final_summary_drop_columns.head()

,election,party,incumbent_challenger_open,receipts,from_individuals,from_committees,from_candidate,disbursements,cash_on_hand,debts,...,american_indian_alaska_native_pop_percent,asian_pop_percent,native_hawaiian_pacific_islander_pop_percent,other_race_pop_percent,hispanic_pop_percent,total_housing_units,spent,won,lost,year
0,2014.0,Republican Party,Incumbent,2643875.43,1137871.17,1443315.73,0.0,2543254.32,1143860.44,15607.71,...,0.0,8.9,0.0,0.1,10.0,270685.0,4079869,1,0,2007.0
1,2014.0,Republican Party,Incumbent,2673006.24,1384157.24,1234898.00,0.0,3084464.90,1472329.77,0.00,...,0.4,1.8,0.1,0.1,27.9,286382.0,3408251,1,0,2003.0
2,2014.0,Republican Party,Incumbent,1425973.57,567158.50,854339.38,0.0,1168222.75,858641.66,0.00,...,1.1,2.0,0.0,0.2,8.8,330578.0,2735431,1,0,2008.0
3,2014.0,Democratic Party,Incumbent,2522320.53,623251.17,1286586.77,0.0,1946781.49,1364191.18,0.00,...,0.0,16.0,0.0,0.4,49.8,262554.0,2668960,1,0,1999.0
4,2014.0,Republican Party,Incumbent,2018388.20,439001.78,1223160.02,0.0,1010201.52,1325444.21,0.00,...,0.1,2.2,0.0,0.0,22.4,290582.0,2184911,1,0,1997.0


In [14]:
# calculate the years in service
final_summary_drop_columns['years_in_service'] = final_summary_drop_columns['election'] - final_summary_drop_columns['year']
final_summary_drop_columns['years_in_service']

0          7.0
1         11.0
2          6.0
3         15.0
4         17.0
         ...  
4872    2018.0
4873    2018.0
4874    2018.0
4875    2018.0
4876    2014.0
Name: years_in_service, Length: 4877, dtype: float64

In [15]:
# Turn new canddiates to 0
years_in_service = final_summary_drop_columns['years_in_service']
final_summary_drop_columns['years_in_service'] = years_in_service[years_in_service > 2000] = 0
final_summary_drop_columns['years_in_service'] = years_in_service[years_in_service < 0] = 0
final_summary_drop_columns.tail()

C:\Users\briordan\Anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\briordan\Anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,election,party,incumbent_challenger_open,receipts,from_individuals,from_committees,from_candidate,disbursements,cash_on_hand,debts,...,asian_pop_percent,native_hawaiian_pacific_islander_pop_percent,other_race_pop_percent,hispanic_pop_percent,total_housing_units,spent,won,lost,year,years_in_service
4872,2018.0,Democratic Party,Open,155331.16,142631.16,9700.0,3000.00,244941.43,1285.31,45864.36,...,19.0,0.2,0.2,63.6,275827.0,0,0,1,0.0,0.0
4873,2018.0,Republican Party,Open,27518.18,6795.67,0.0,20722.51,30835.17,-3316.99,15158.99,...,3.6,0.0,0.1,30.3,358577.0,0,0,1,0.0,0.0
4874,2018.0,Democratic Party,Open,74353.87,73493.85,0.0,860.02,74353.87,0.00,0.00,...,9.6,0.8,0.3,12.1,297606.0,0,0,1,0.0,0.0
4875,2018.0,Democratic Party,Challenger,57777.75,29205.37,500.0,28072.38,54737.28,3090.47,0.00,...,1.5,0.0,0.2,2.0,325777.0,0,0,1,0.0,0.0
4876,2014.0,Democratic Party,Challenger,1000.00,1000.00,0.0,0.00,1000.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,0,0,1,0.0,0.0


In [16]:
final_summary_drop_columns = final_summary_drop_columns.drop(columns=["year"])

In [17]:
# final_summary_drop_columns.to_csv('final_table_1.csv')

In [18]:
# Creating dummy data
data_with_dummies = pd.get_dummies(final_summary_drop_columns, columns=["party","incumbent_challenger_open"])
data_with_dummies.head()

,election,receipts,from_individuals,from_committees,from_candidate,disbursements,cash_on_hand,debts,total_pop,male_pop_percent,...,spent,won,lost,years_in_service,party_Democratic Party,party_Other,party_Republican Party,incumbent_challenger_open_Challenger,incumbent_challenger_open_Incumbent,incumbent_challenger_open_Open
0,2014.0,2643875.43,1137871.17,1443315.73,0.0,2543254.32,1143860.44,15607.71,727718.0,49.2,...,4079869,1,0,7.0,0,0,1,0,1,0
1,2014.0,2673006.24,1384157.24,1234898.00,0.0,3084464.90,1472329.77,0.00,726896.0,50.4,...,3408251,1,0,11.0,0,0,1,0,1,0
2,2014.0,1425973.57,567158.50,854339.38,0.0,1168222.75,858641.66,0.00,781134.0,48.6,...,2735431,1,0,6.0,0,0,1,0,1,0
3,2014.0,2522320.53,623251.17,1286586.77,0.0,1946781.49,1364191.18,0.00,727544.0,51.3,...,2668960,1,0,15.0,1,0,0,0,1,0
4,2014.0,2018388.20,439001.78,1223160.02,0.0,1010201.52,1325444.21,0.00,761764.0,51.0,...,2184911,1,0,17.0,0,0,1,0,1,0


In [19]:
# data_with_dummies.to_csv('final_table_1.csv')

In [20]:
# Drop the state_district
data_with_dummies = data_with_dummies.drop(columns=['lost'])
data_with_dummies.head()

,election,receipts,from_individuals,from_committees,from_candidate,disbursements,cash_on_hand,debts,total_pop,male_pop_percent,...,total_housing_units,spent,won,years_in_service,party_Democratic Party,party_Other,party_Republican Party,incumbent_challenger_open_Challenger,incumbent_challenger_open_Incumbent,incumbent_challenger_open_Open
0,2014.0,2643875.43,1137871.17,1443315.73,0.0,2543254.32,1143860.44,15607.71,727718.0,49.2,...,270685.0,4079869,1,7.0,0,0,1,0,1,0
1,2014.0,2673006.24,1384157.24,1234898.00,0.0,3084464.90,1472329.77,0.00,726896.0,50.4,...,286382.0,3408251,1,11.0,0,0,1,0,1,0
2,2014.0,1425973.57,567158.50,854339.38,0.0,1168222.75,858641.66,0.00,781134.0,48.6,...,330578.0,2735431,1,6.0,0,0,1,0,1,0
3,2014.0,2522320.53,623251.17,1286586.77,0.0,1946781.49,1364191.18,0.00,727544.0,51.3,...,262554.0,2668960,1,15.0,1,0,0,0,1,0
4,2014.0,2018388.20,439001.78,1223160.02,0.0,1010201.52,1325444.21,0.00,761764.0,51.0,...,290582.0,2184911,1,17.0,0,0,1,0,1,0


In [21]:
data_with_dummies.columns

Index(['election', 'receipts', 'from_individuals', 'from_committees',
       'from_candidate', 'disbursements', 'cash_on_hand', 'debts', 'total_pop',
       'male_pop_percent', 'female_pop_percent', 'under_five_percent',
       'five_to_nine_percent', 'ten_to_fourteen_percent',
       'fifteen_nineteen_percent', 'twenty_to_twentyfour_percent',
       'twentyfive_to_thirtyfour_percent', 'thirtyfive_to_fortyfour_percent',
       'fortyfive_to_fiftyfour_percent', 'fiftyfive_to_fiftynine_percent',
       'sixty_to_sixtyfour_percent', 'sixtyfive_to_seventyfour_percent',
       'seventyfive_to_eightyfour_percent', 'over_eightyfive_percent',
       'median_age', 'eighteen_and_over_percent', 'twentyone_and_over_percent',
       'sixtytwo_and_over_percent', 'sixtyfive_and_over_percent',
       'eighteen_and_over_male_percent', 'eighteen_and_over_female_percent',
       'sixtyfive_and_over_male_percent', 'sixtyfive_and_over_female_percent',
       'white_pop_percent', 'black_pop_percent',
      

In [22]:
# Fill NaN with 0
data_with_dummies = data_with_dummies.fillna(0)

In [23]:
data_with_dummies['one_to_nineteen_percent'] = data_with_dummies['under_five_percent'] + data_with_dummies['five_to_nine_percent'] +  data_with_dummies['ten_to_fourteen_percent'] + data_with_dummies['fifteen_nineteen_percent'] 
data_with_dummies['twenty_to_fiftyfour_percent'] = data_with_dummies['twenty_to_twentyfour_percent'] + data_with_dummies['twentyfive_to_thirtyfour_percent'] + data_with_dummies['thirtyfive_to_fortyfour_percent'] + data_with_dummies['fortyfive_to_fiftyfour_percent']
data_with_dummies['fiftyfive_to_100_percent'] = data_with_dummies['fiftyfive_to_fiftynine_percent'] + data_with_dummies['sixty_to_sixtyfour_percent'] + data_with_dummies['sixtyfive_to_seventyfour_percent'] + data_with_dummies['seventyfive_to_eightyfour_percent'] + data_with_dummies['over_eightyfive_percent']

In [24]:
data_with_dummies = data_with_dummies.drop(columns=['under_five_percent','five_to_nine_percent', 'ten_to_fourteen_percent', 'fifteen_nineteen_percent', 
                                                   'twenty_to_twentyfour_percent', 'twentyfive_to_thirtyfour_percent', 'thirtyfive_to_fortyfour_percent', 'fortyfive_to_fiftyfour_percent',
                                                   'fiftyfive_to_fiftynine_percent', 'sixty_to_sixtyfour_percent', 'sixtyfive_to_seventyfour_percent', 'seventyfive_to_eightyfour_percent',
                                                   'over_eightyfive_percent'])

In [96]:
# Create the y and X labels
# 'from_committees', 'disbursements', 'from_individuals'
y = data_with_dummies["won"]
X = data_with_dummies.drop(columns=['won'])
X.head()

,election,receipts,from_individuals,from_committees,from_candidate,disbursements,cash_on_hand,debts,total_pop,male_pop_percent,...,years_in_service,party_Democratic Party,party_Other,party_Republican Party,incumbent_challenger_open_Challenger,incumbent_challenger_open_Incumbent,incumbent_challenger_open_Open,one_to_nineteen_percent,twenty_to_fiftyfour_percent,fiftyfive_to_100_percent
0,2014.0,2643875.43,1137871.17,1443315.73,0.0,2543254.32,1143860.44,15607.71,727718.0,49.2,...,7.0,0,0,1,0,1,0,27.4,45.7,26.8
1,2014.0,2673006.24,1384157.24,1234898.00,0.0,3084464.90,1472329.77,0.00,726896.0,50.4,...,11.0,0,0,1,0,1,0,28.6,47.2,24.1
2,2014.0,1425973.57,567158.50,854339.38,0.0,1168222.75,858641.66,0.00,781134.0,48.6,...,6.0,0,0,1,0,1,0,25.6,47.0,27.6
3,2014.0,2522320.53,623251.17,1286586.77,0.0,1946781.49,1364191.18,0.00,727544.0,51.3,...,15.0,1,0,0,0,1,0,22.4,54.7,22.9
4,2014.0,2018388.20,439001.78,1223160.02,0.0,1010201.52,1325444.21,0.00,761764.0,51.0,...,17.0,0,0,1,0,1,0,29.0,46.8,24.3


In [97]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [98]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [99]:
# Run random forest with cross validation
from sklearn.model_selection import cross_validate
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78, min_sample_leaf=2)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)


In [100]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [101]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,878,27
Actual 1,23,292


In [102]:
cv = cross_validate(rf_model, X, y, cv=5)
print(cv['test_score'])

[0.94262295 0.91905738 0.90051282 0.98769231 0.91179487]


In [103]:
# Classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.97      0.97      0.93      0.97      0.95      0.90       905
          1       0.92      0.93      0.97      0.92      0.95      0.90       315

avg / total       0.96      0.96      0.94      0.96      0.95      0.90      1220



In [104]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.25818374315274084, 'spent'),
 (0.1288445637445859, 'years_in_service'),
 (0.10945654074866902, 'incumbent_challenger_open_Incumbent'),
 (0.1055352615963174, 'from_committees'),
 (0.08546023000467608, 'cash_on_hand'),
 (0.05426334900481465, 'receipts'),
 (0.0500516247299731, 'disbursements'),
 (0.034998230032800844, 'from_individuals'),
 (0.03154421005458919, 'incumbent_challenger_open_Challenger'),
 (0.0136324394855204, 'from_candidate'),
 (0.006868795230728395, 'asian_pop_percent'),
 (0.006641537145234548, 'total_pop'),
 (0.006465070114631025, 'incumbent_challenger_open_Open'),
 (0.006426962749578042, 'white_pop_percent'),
 (0.005860372629652951, 'black_pop_percent'),
 (0.0054470945738824775, 'debts'),
 (0.0053663272297302675, 'hispanic_pop_percent'),
 (0.00534539130244353, 'twenty_to_fiftyfour_percent'),
 (0.005172694029508705, 'total_housing_units'),
 (0.004980666523734993, 'sixtyfive_and_over_male_percent'),
 (0.004744166049944518, 'fiftyfive_to_100_percent'),
 (0.0046589078654

In [105]:
# Find Correlations
data_with_dummies.corr()

,election,receipts,from_individuals,from_committees,from_candidate,disbursements,cash_on_hand,debts,total_pop,male_pop_percent,...,years_in_service,party_Democratic Party,party_Other,party_Republican Party,incumbent_challenger_open_Challenger,incumbent_challenger_open_Incumbent,incumbent_challenger_open_Open,one_to_nineteen_percent,twenty_to_fiftyfour_percent,fiftyfive_to_100_percent
election,1.000000,-0.004433,0.008723,-0.064908,0.018248,0.007475,-0.010600,-0.013769,0.218717,0.053568,...,-0.068424,0.125162,0.007549,-0.128802,0.023590,-0.089174,0.060452,-0.111456,-0.171008,0.201823
receipts,-0.004433,1.000000,0.842214,0.600738,0.358690,0.923252,0.649613,0.216930,0.007152,0.021628,...,0.259760,0.003249,-0.102884,0.042671,-0.317439,0.427788,-0.058865,-0.013202,-0.000918,0.011200
from_individuals,0.008723,0.842214,1.000000,0.378102,0.020894,0.772034,0.483680,0.050553,0.024047,0.020941,...,0.139696,0.058278,-0.086388,-0.019840,-0.218559,0.274347,-0.020801,-0.030978,0.005969,0.016793
from_committees,-0.064908,0.600738,0.378102,1.000000,-0.063215,0.434220,0.741022,-0.013006,-0.044562,0.010211,...,0.517531,-0.064708,-0.104070,0.111303,-0.447366,0.771977,-0.248206,0.036031,0.009646,-0.027346
from_candidate,0.018248,0.358690,0.020894,-0.063215,1.000000,0.414862,0.006832,0.499757,0.013167,0.004179,...,-0.054336,-0.009156,-0.021976,0.018986,-0.037707,-0.076788,0.117707,-0.016868,-0.016993,0.023831
disbursements,0.007475,0.923252,0.772034,0.434220,0.414862,1.000000,0.411772,0.202365,0.024849,0.010479,...,0.233867,0.005148,-0.085284,0.032911,-0.261568,0.323110,-0.019788,-0.009329,0.022732,-0.009076
cash_on_hand,-0.010600,0.649613,0.483680,0.741022,0.006832,0.411772,1.000000,0.067454,-0.032252,0.016761,...,0.390548,-0.016507,-0.101103,0.061674,-0.361157,0.610706,-0.188153,-0.017047,-0.006290,0.017208
debts,-0.013769,0.216930,0.050553,-0.013006,0.499757,0.202365,0.067454,1.000000,0.002587,-0.003381,...,-0.055258,-0.061511,-0.033967,0.076806,-0.062807,-0.013844,0.084596,-0.021208,-0.032413,0.037997
total_pop,0.218717,0.007152,0.024047,-0.044562,0.013167,0.024849,-0.032252,0.002587,1.000000,0.128137,...,-0.007654,0.037707,-0.029225,-0.024742,0.016894,-0.049902,0.029651,0.160809,0.178623,-0.166530
male_pop_percent,0.053568,0.021628,0.020941,0.010211,0.004179,0.010479,0.016761,-0.003381,0.128137,1.000000,...,-0.039404,-0.049354,0.041131,0.031100,0.025180,-0.036287,0.006969,0.224831,0.146462,-0.064613


In [106]:
# Find all correlations greater than .7
rows, cols = data_with_dummies.shape
flds = list(data_with_dummies.columns)
corr = data_with_dummies.corr().values

for i in range(cols):
    for j in range(i+1, cols):
        if corr[i,j] > 0.7:
            print(flds[i], ' ', flds[j], ' ', corr[i,j])
        elif corr[i,j] < -0.7:
            print(flds[i], ' ', flds[j], ' ', corr[i,j])

receipts   from_individuals   0.8422140317193026
receipts   disbursements   0.9232524330275558
from_individuals   disbursements   0.7720338727512792
from_committees   cash_on_hand   0.7410217916329974
from_committees   incumbent_challenger_open_Incumbent   0.771976892280303
male_pop_percent   eighteen_and_over_male_percent   0.9765424993188658
female_pop_percent   eighteen_and_over_female_percent   0.9778358765068297
median_age   twentyone_and_over_percent   0.7510927085390107
median_age   sixtytwo_and_over_percent   0.904505292317952
median_age   sixtyfive_and_over_percent   0.8956552608566984
median_age   fiftyfive_to_100_percent   0.9341258729774409
eighteen_and_over_percent   twentyone_and_over_percent   0.9626509405805096
eighteen_and_over_percent   one_to_nineteen_percent   -0.8599818073153489
twentyone_and_over_percent   one_to_nineteen_percent   -0.8907939715609831
sixtytwo_and_over_percent   sixtyfive_and_over_percent   0.9967571645445905
sixtytwo_and_over_percent   twenty_to_

In [131]:
# Create a randomized Hyperparameter Grid
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)


{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [132]:
# Create the y and X labels
y = data_with_dummies["won"]
X = data_with_dummies[['years_in_service', 'spent', 'incumbent_challenger_open_Incumbent', 'receipts', 'cash_on_hand', 'incumbent_challenger_open_Challenger', 'from_candidate']]
X.head()

,years_in_service,spent,incumbent_challenger_open_Incumbent,receipts,cash_on_hand,incumbent_challenger_open_Challenger,from_candidate
0,7.0,4079869,1,2643875.43,1143860.44,0,0.0
1,11.0,3408251,1,2673006.24,1472329.77,0,0.0
2,6.0,2735431,1,1425973.57,858641.66,0,0.0
3,15.0,2668960,1,2522320.53,1364191.18,0,0.0
4,17.0,2184911,1,2018388.20,1325444.21,0,0.0


In [133]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [134]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [135]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=400, random_state=78, bootstrap = True)

In [136]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [137]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [138]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,876,29
Actual 1,13,302


In [139]:
# Classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.99      0.97      0.96      0.98      0.96      0.93       905
          1       0.91      0.96      0.97      0.93      0.96      0.93       315

avg / total       0.97      0.97      0.96      0.97      0.96      0.93      1220



In [140]:
cv = cross_validate(rf_model, X, y, cv=5)
print(cv['test_score'])

[0.96004098 0.95184426 0.92307692 0.98871795 0.94358974]


In [141]:
# Sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.35885465974569014, 'spent'),
 (0.23354369717903606, 'years_in_service'),
 (0.14193074581636544, 'incumbent_challenger_open_Incumbent'),
 (0.11213841111125665, 'cash_on_hand'),
 (0.09479699353125844, 'receipts'),
 (0.033229554166567594, 'incumbent_challenger_open_Challenger'),
 (0.025505938449825644, 'from_candidate')]

In [149]:
rf_random = RandomizedSearchCV(RandomForestClassifier(n_estimators=400),param_distributions = random_grid, 
                               n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 11.0min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 14.8min finished


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(n_estimators=400),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [150]:
rf_random.best_params_

{'n_estimators': 800,
 'min_samples_split': 10,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 100,
 'bootstrap': True}

In [151]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [153]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=800, random_state=42, min_samples_split=10, min_samples_leaf=4, 
                                  max_features='sqrt', max_depth=100, bootstrap = True)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [154]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,879,34
Actual 1,13,294


In [155]:
# Classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.99      0.96      0.96      0.97      0.96      0.92       913
          1       0.90      0.96      0.96      0.93      0.96      0.92       307

avg / total       0.96      0.96      0.96      0.96      0.96      0.92      1220



In [156]:
cv = cross_validate(rf_model, X, y, cv=5)
print(cv['test_score'])

[0.96106557 0.95286885 0.92512821 0.99076923 0.94051282]


In [157]:
rf_random = RandomizedSearchCV(RandomForestClassifier(n_estimators=400),param_distributions = random_grid, 
                               n_iter = 100, cv = 5, verbose=2, random_state=75, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   46.4s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 10.8min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 14.2min finished


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(n_estimators=400),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=75, verbose=2)

In [158]:
rf_random.best_params_

{'n_estimators': 1000,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 100,
 'bootstrap': True}

In [159]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=75)

In [160]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=1000, random_state=75, min_samples_split=2, min_samples_leaf=4, 
                                  max_features='sqrt', max_depth=100, bootstrap = True)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [161]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,910,23
Actual 1,17,270


In [162]:
# Classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.98      0.98      0.94      0.98      0.96      0.92       933
          1       0.92      0.94      0.98      0.93      0.96      0.91       287

avg / total       0.97      0.97      0.95      0.97      0.96      0.92      1220



In [163]:
cv = cross_validate(rf_model, X, y, cv=5)
print(cv['test_score'])

[0.95901639 0.95491803 0.92410256 0.99076923 0.94051282]


In [89]:
# import seaborn as sns

In [ ]:
# import seaborn as sns; sns.set(style="ticks", color_codes=True)
# g = sns.pairplot(iris)

In [ ]:
# sns.set(style="ticks", color_codes=True)
# sns.pairplot(X)

In [ ]:
# fig = sns.save_figure()
# sns_plot1.savefig("output.png")

In [ ]:
# check for overfitting

In [ ]:
# Bagging/Bootstrap